In [1]:
import pandas as pd
import numpy as np

In [2]:
# session_based embedding
df_deal_vec = pd.read_csv('df_item_sbwin5_trainset.csv.gz', compression='gzip')
df_user_vec = pd.read_csv('df_user_sbwin5_trainset.csv.gz', compression='gzip')

In [2]:
# content_based embedding
df_deal_vec = pd.read_csv('df_item_cbwin5_trainset.csv.gz', compression='gzip')
df_user_vec = pd.read_csv('df_user_cbwin5_trainset.csv.gz', compression='gzip')

In [3]:
df_deal_vec = df_deal_vec[['deal_id', 'dealvec']]
df_user_vec = df_user_vec[['account_id', 'uservec']]

In [5]:
# session_based embedding
df_deal_vec.dealvec = df_deal_vec.dealvec.map(lambda x: x[1:-1].split())

In [4]:
# content_based embedding
df_deal_vec.dealvec = df_deal_vec.dealvec.map(lambda x: x[2:-2].split())

In [5]:
df_user_vec.uservec = df_user_vec.uservec.map(lambda x: x[2:-2].split())

In [6]:
for i in range(200):
    df_deal_vec['dealvec_' + str(i)] = df_deal_vec.dealvec.map(lambda x: float(x[i]))
for i in range(200):
    df_user_vec['uservec_' + str(i)] = df_user_vec.uservec.map(lambda x: float(x[i]))
df_deal_vec.deal_id = df_deal_vec.deal_id.astype('category').cat.codes.values
df_user_vec.account_id = df_user_vec.account_id.astype('category').cat.codes.values

In [12]:
# random split train_test
deal_order = pd.read_csv('/home/mmde-lab/lizhi/Naive RS/' + 'df_order_add_negasmpl.gz', compression = 'gzip')
deal_order = deal_order[['account_id', 'deal_id', 'rating']]
df_order_after = pd.merge(deal_order, df_deal_vec, how='left', left_on=['deal_id'], right_on=['deal_id'])
df_order_after = pd.merge(df_order_after, df_user_vec, how='left', left_on=['account_id'], right_on=['account_id'])
df_order_after = df_order_after.dropna()
from sklearn.model_selection import train_test_split
train_order_after1, test_order_after1 = train_test_split(df_order_after, test_size = 0.1)
train_order_after1, valid_order_after1 = train_test_split(train_order_after1, test_size = 0.2)

In [7]:
# split train_test by time
work_path = '/home/mmde-lab/lizhi/Naive RS/'
train_order = pd.read_csv(work_path + 'train_order_add_negasmpl.gz', compression='gzip')
valid_order = pd.read_csv(work_path + 'valid_order_add_negasmpl.gz', compression='gzip')
test_order = pd.read_csv(work_path + 'test_order_add_negasmpl.gz', compression='gzip')

train_order = train_order[['account_id', 'deal_id', 'rating']]
valid_order = valid_order[['account_id', 'deal_id', 'rating']]
test_order = test_order[['account_id', 'deal_id', 'rating']]

train_order_after1 = pd.merge(train_order, df_deal_vec, how='left', left_on=['deal_id'], right_on=['deal_id'])
valid_order_after1 = pd.merge(valid_order, df_deal_vec, how='left', left_on=['deal_id'], right_on=['deal_id'])
test_order_after1 = pd.merge(test_order, df_deal_vec, how='left', left_on=['deal_id'], right_on=['deal_id'])

train_order_after1 = pd.merge(train_order_after1, df_user_vec, how='left', left_on=['account_id'], right_on=['account_id'])
valid_order_after1 = pd.merge(valid_order_after1, df_user_vec, how='left', left_on=['account_id'], right_on=['account_id'])
test_order_after1 = pd.merge(test_order_after1, df_user_vec, how='left', left_on=['account_id'], right_on=['account_id'])

train_order_after1.shape, valid_order_after1.shape, test_order_after1.shape
train_order_after1 = train_order_after1.dropna()
valid_order_after1 = valid_order_after1.dropna()
test_order_after1 = test_order_after1.dropna()

/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
train_order_after1.shape, valid_order_after1.shape, test_order_after1.shape
user_vec_name = []
deal_vec_name = []
for i in range(200):
    user_vec_name.append('uservec_' + str(i))
    deal_vec_name.append('dealvec_' + str(i))

# GBDT model

In [9]:
def learning_rate_power(current_round):
    base_learning_rate = 0.19000424246380565
    min_learning_rate = 0.01
    lr = base_learning_rate * np.power(0.995,current_round)
    return max(lr, min_learning_rate)

In [19]:
import lightgbm as lgb
lgb_clf = lgb.LGBMClassifier(n_jobs=20, objective='binary',random_state=1)
opt_params = {'n_estimators':1000,
              'boosting_type': 'GBDT',
              'objective': 'binary',
              'num_leaves': 128,
              'min_child_samples':212,
              'reg_lambda':0.01}
fit_params = {'early_stopping_rounds':10,
              'eval_metric': 'accuracy',
              'eval_set': (valid_order_after1.loc[:, user_vec_name + deal_vec_name], valid_order_after1.rating),
              'verbose': 2,
              'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_power)]}
lgb_clf.set_params(**opt_params)
lgb_clf.fit(train_order_after1.loc[:, user_vec_name + deal_vec_name], train_order_after1.rating,**fit_params)

Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.645353
[4]	valid_0's binary_logloss: 0.623153
[6]	valid_0's binary_logloss: 0.595808
[8]	valid_0's binary_logloss: 0.575342
[10]	valid_0's binary_logloss: 0.562894
[12]	valid_0's binary_logloss: 0.551588
[14]	valid_0's binary_logloss: 0.543416
[16]	valid_0's binary_logloss: 0.537254
[18]	valid_0's binary_logloss: 0.529749
[20]	valid_0's binary_logloss: 0.525646
[22]	valid_0's binary_logloss: 0.522068
[24]	valid_0's binary_logloss: 0.517909
[26]	valid_0's binary_logloss: 0.516
[28]	valid_0's binary_logloss: 0.513697
[30]	valid_0's binary_logloss: 0.510679
[32]	valid_0's binary_logloss: 0.508705
[34]	valid_0's binary_logloss: 0.505732
[36]	valid_0's binary_logloss: 0.50437
[38]	valid_0's binary_logloss: 0.502283
[40]	valid_0's binary_logloss: 0.499277
[42]	valid_0's binary_logloss: 0.498306
[44]	valid_0's binary_logloss: 0.496583
[46]	valid_0's binary_logloss: 0.495919
[48]	valid_0's binary_logl

LGBMClassifier(boosting_type='GBDT', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=212, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=1000, n_jobs=20, num_leaves=128, objective='binary',
        random_state=1, reg_alpha=0.0, reg_lambda=0.01, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [20]:
from sklearn.metrics import accuracy_score

In [22]:
print('Training accuracy: ', accuracy_score(train_order_after1.rating, lgb_clf.predict(train_order_after1.loc[:, user_vec_name + deal_vec_name])))
print('Validation accuracy: ', accuracy_score(valid_order_after1.rating, lgb_clf.predict(valid_order_after1.loc[:, user_vec_name + deal_vec_name])))
print('Test accuracy: ', accuracy_score(test_order_after1.rating, lgb_clf.predict(test_order_after1.loc[:, user_vec_name + deal_vec_name])))

/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Training accuracy:  0.7962504197851659
Validation accuracy:  0.7595964243459971
Test accuracy:  0.6803648288583726


/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [37]:
fit_params = {'early_stopping_rounds':10,
              'eval_metric': 'accuracy',
              'eval_set': (test_order_after1.loc[:, user_vec_name + deal_vec_name], test_order_after1.rating),
              'verbose': 2,
              'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_power)]}
lgb_clf.fit(train_order_after1.loc[:, user_vec_name + deal_vec_name].append(valid_order_after1.loc[:, user_vec_name + deal_vec_name]),
            train_order_after1.rating.append(valid_order_after1.rating),**fit_params)

Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.582983
[4]	valid_0's binary_logloss: 0.540094
[6]	valid_0's binary_logloss: 0.515747
[8]	valid_0's binary_logloss: 0.504752
[10]	valid_0's binary_logloss: 0.501868
[12]	valid_0's binary_logloss: 0.502893
[14]	valid_0's binary_logloss: 0.50477
[16]	valid_0's binary_logloss: 0.503074
[18]	valid_0's binary_logloss: 0.504503
[20]	valid_0's binary_logloss: 0.507879
Early stopping, best iteration is:
[11]	valid_0's binary_logloss: 0.501463


LGBMClassifier(boosting_type='GBDT', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=212, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=1000, n_jobs=20, num_leaves=2452, objective='binary',
        random_state=1, reg_alpha=0.0, reg_lambda=0.01, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [14]:
print('Training accuracy: ', accuracy_score(train_order_after1.rating, lgb_clf.predict(train_order_after1.loc[:, user_vec_name + deal_vec_name])))
print('Validation accuracy: ', accuracy_score(valid_order_after1.rating, lgb_clf.predict(valid_order_after1.loc[:, user_vec_name + deal_vec_name])))
print('Test accuracy: ', accuracy_score(test_order_after1.rating, lgb_clf.predict(test_order_after1.loc[:, user_vec_name + deal_vec_name])))

/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Training accuracy:  0.7962504197851659
Validation accuracy:  0.7595964243459971
Test accuracy:  0.6803648288583726


/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# num_leaves Training,  Validation,  Test accuracy
## 2452  0.9995961322345008  0.8859571080726728  0.885279132197447
## 128  0.8705369685334332 0.8538771305487919  0.8530652495480195 
##  512  0.9281548510957108  0.8754495223824686  0.874815098887854
##  1024 0.9683391084472748  0.8812839483049261  0.8805844350138439

# num_leaves Training,  Validation,  Test accuracy     (trainset_embedding)
## 2452  0.8309327426677791  0.7795451557775733  0.6574093051854888
## 1024  0.8273837007365532  0.774582621269883  0.6569993851199016
## 512   0.820963663779482  0.7741882476666229  0.6592539454806313